<a href="https://colab.research.google.com/github/paladugulashruthi/NNDL/blob/main/assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
import seaborn as sns
from scipy import stats

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/kc_house_data.csv")
data.head()

In [ ]:
z = np.abs(stats.zscore(data['price']))
print(z)

In [ ]:
sns.boxplot(x=data['price'])

In [2]:
df = pd.DataFrame(data=data)
df.head()

NameError: ignored

In [ ]:
x = df
df.drop(['date'],axis=1)
for i in df:
    if i != 'date':
        q = df[i].quantile(0.99)
        df[df[i] < q]
        q_low = df[i].quantile(0.01)
        q_hi  = df[i].quantile(0.99)

        df_filtered = df[(df[i] < q_hi) & (df[i] > q_low)]
        
df_filtered.head()

In [ ]:
print(len(df))
print(len(df_filtered))
if(len(df) == len(df_filtered)):
    print('No outliners')
data = df_filtered

In [ ]:
new_data = data.copy()
for i in data:
    if i!='date':
        new_data[i] = data[i].copy()
        new_data[i] = (new_data[i] -new_data[i].min())/(new_data[i].max() - new_data[i].min())
new_data.head()

In [ ]:
train_data,test_data = train_test_split(new_data,train_size=0.9)
print(len(train_data))
train_data.head()

In [ ]:
x_val = np.array(train_data['sqft_living'])
y_val = np.array(train_data['price'])

In [ ]:
from sympy import Symbol, Derivative, symbols
m= Symbol('m')
c= Symbol('c')
function = '0.5*(y-m*x-c)**2'
partialderiv= Derivative(function, m)
dfm = partialderiv.doit()
partialderiv= Derivative(function, c)
dfc = partialderiv.doit()

print(f'first order derivatives of given function are de/dm = {dfm} ,de/dc = {dfc}')
err = []
#step 1 initialise values
bs = 2
m1 = 1.0 
c1 = -1.0  #variable x, y
epoches=5
learning_rate = 0.1  #learning rate

ns = len(x_val)
nb = ns//bs
ms = ns%bs


for i in range(0,epoches):
    for k in range(0,nb):
        er = 0
        dfmv1 = 0
        dfcv1 = 0
        for j in range(0,bs):
            er +=  ((y_val[j] - (m1 * x_val[j]) - c1))**2
            index = k*bs + j
            #step 2 substitute x ,y in first order derivatives df/dx, df/dy
            m = symbols('m')
            c = symbols('c')
            x = symbols('x')
            y = symbols('y')
            dfmv = dfm.subs(m, m1) 
            dfmv = dfmv.subs(c, c1) 
            dfmv = dfmv.subs(x, x_val[index]) 
            dfmv = dfmv.subs(y, y_val[index]) 
            dfmv = round(dfmv,2)
            dfcv = dfc.subs(c, c1) 
            dfcv = dfcv.subs(m, m1) 
            dfcv = dfcv.subs(x, x_val[index]) 
            dfcv = dfcv.subs(y, y_val[index]) 
            dfcv = round(dfcv,2)
            dfmv1 += dfmv
            dfcv1 += dfcv
            
        dfmv1 = dfmv1/bs
        dfcv1 = dfcv1/bs

        dfmv1 = round(dfmv1,2)
        dfcv1 = round(dfcv1,2)
        #step3 find change in x , y
        dm = (-1.0)*learning_rate*dfmv1
        dc = (-1.0)*learning_rate*dfcv1
        #step4 update variable 
        m1 = m1 + dm 
        c1 = c1 + dc
        
        er = (1/(2*len(train_data['sqft_living'])))*(er)
        err.append(er)
        #step5 increment iterations 
        #step6 break loop if iterations exceed no of epoches
k = nb
if(ms>0):
    er = 0
    dfmv1 = 0
    dfcv1 = 0
    for j in range(0,ms):
        
        er +=  ((ya[j] - (m1 * xa[j]) - c1))**2
        index = k*bs + j
        #step 2 substitute x ,y in first order derivatives df/dx, df/dy
        m = symbols('m')
        c = symbols('c')
        x = symbols('x')
        y = symbols('y')
        dfmv = dfm.subs(m, m1) 
        dfmv = dfmv.subs(c, c1) 
        dfmv = dfmv.subs(x, xa[index]) 
        dfmv = dfmv.subs(y, ya[index]) 
        dfcv = dfc.subs(c, c1) 
        dfcv = dfcv.subs(m, m1) 
        dfcv = dfcv.subs(x, xa[index]) 
        dfcv = dfcv.subs(y, ya[index]) 
        dfmv1 += dfmv
        dfcv1 += dfcv

    dfmv1 = dfmv1/ms
    dfcv1 = dfcv1/ms

    #step3 find change in x , y
    dm = (-1.0)*learning_rate*dfmv1
    dc = (-1.0)*learning_rate*dfcv1
    #step4 update variable 
    m1 = m1 + dm 
    c1 = c1 + dc
    
    er = (1/(2*len(train_data['sqft_living'])))*(er)
    err.append(er)
#step7 print variable x1, y1
m1= round(m1,2)
c1= round(c1,2)
print(m1, c1)
print(f'minimum value obtained at m = {m1} ,c ={c1} for given function')


In [ ]:
train_data_pred = [i for i in x_val]
for k in range(len(x_val)):
    train_data_pred[k]=(m1*x_val[k] + c1)
from sklearn.metrics import mean_squared_error
train_mse = mean_squared_error(y_val,train_data_pred)
train_mse

In [ ]:
plt.plot(np.array(train_data['sqft_living']),np.array(train_data_pred),'red')
plt.scatter(np.array(train_data['sqft_living']),np.array(train_data['price']),color='cyan')
plt.title('predicted data vs actual data')

In [ ]:
iters = np.arange(epoches*nb)
plt.plot(iters,err,'pink')
plt.title("Error graph for training data")
plt.xlabel("iteration count")
plt.ylabel("error")
plt.grid()

In [ ]:
xa = np.array(test_data['sqft_living'])
ya = np.array(test_data['price'])

test_data_pred = [i for i in xa]
for k in range(len(xa)):
    test_data_pred[k]=(m1*xa[k] + c1)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(ya,test_data_pred)
mse

In [ ]:
plt.plot(test_data['price'],test_data_pred,'green')
plt.grid()

In [ ]:
x_min = data['sqft_living'].min()
x_max = data['sqft_living'].max()
y_min = data['price'].min()
y_max = data['price'].max()

In [ ]:
x1 = float(input("Enter sqrt feet values sqft_living , sqft_lot, sqft_above, sqft_basement, sqft_living15, sqft_lot15 value :"))

x1 = (x1 - x_min)/(x_max - x_min)
y = m1*x1+c1
y = y*(y_max - y_min) + y_min
print(f"price  is : {y}")